In [0]:
df_population_csv = (spark.read
    .format("csv")
    .option("header", True)        # primera fila como nombres de columna
    .option("inferSchema", True)   # inferir tipos automáticamente
    .option("nullValue", "")    # opcional: tratar cadenas vacías como null
    .load("/Volumes/workspace/default/workshop/population_data.csv")
)

df_population_csv.display()

In [0]:
df_population_csv = (spark.read
    .format("csv")
    .option("header", True)        # primera fila como nombres de columna
    .option("inferSchema", True)   # inferir tipos automáticamente
    .option("nullValue", "")
    .option("skipRows", 3)       # opcional: tratar cadenas vacías como null
    .load("/Volumes/workspace/default/workshop/population_data.csv")
)

In [0]:
df_population_csv = df_population_csv.select("Country Name", "Country Code")
display(df_population_csv)

In [0]:
df_population_json = (spark.read
    .format("json")
    .load("/Volumes/workspace/default/workshop/population_data.txt")
)

In [0]:
df_population_json.display()

In [0]:
from pyspark.sql.functions import expr, map_from_entries, col

# 1. Lee el XML como ya lo tienes:
df_xml = (spark.read
    .format("xml")
    .option("rowTag", "record")
    .load("/Volumes/workspace/default/workshop/population_data.xml")
)

In [0]:
df_xml.display()

In [0]:
# 2. Convierte el array<struct<_name, _VALUE>> en un map<name, value>
#    transformamos cada elemento x en un struct(key=x._name, value=x._VALUE)
df_map = df_xml.select(
    map_from_entries(
       expr("transform(field, x -> struct(x._name, x._VALUE))")
    ).alias("m")
)

In [0]:
df_map.display()

In [0]:
# 3. Extrae cada clave del map como columna
df_final = df_map.select(
    col("m.`Country or Area`").alias("country"),
    col("m.Item").alias("indicator"),
    col("m.Year").cast("int").alias("year"),
    col("m.Value").cast("long").alias("population")
)

df_final.display()


In [0]:
import sqlite3
import pandas as pd

db_path = "/Volumes/workspace/default/workshop/population_data.db"
conn    = sqlite3.connect(db_path)

# 2. Lee una tabla o haz tu query
query   = "SELECT * FROM population_data;" 
pdf     = pd.read_sql_query(query, conn)

# 3. Convierte a Spark DataFrame
df = spark.createDataFrame(pdf)

# 4. Usa tu DataFrame en PySpark
df.display()

## Combine Data

In [0]:
df_rural = (spark.read
    .format("csv")
    .option("header", True)        # primera fila como nombres de columna
    .option("inferSchema", True)   # inferir tipos automáticamente
    .option("nullValue", "")
    .option("skipRows", 3)       # opcional: tratar cadenas vacías como null
    .load("/Volumes/workspace/default/workshop/rural_population_percent.csv")
) 

In [0]:
df_rural.display()

In [0]:
df_electricity = (spark.read
    .format("csv")
    .option("header", True)        # primera fila como nombres de columna
    .option("inferSchema", True)   # inferir tipos automáticamente
    .option("nullValue", "")
    .option("skipRows", 3)       # opcional: tratar cadenas vacías como null
    .load("/Volumes/workspace/default/workshop/electricity_access_percent.csv")
) 

In [0]:
df_electricity.display()

In [0]:
df_concatenado = df_rural.unionByName(df_electricity)

In [0]:
df_concatenado.display()

In [0]:
import requests
import pandas as pd
from pyspark.sql.functions import col

url = 'http://api.worldbank.org/v2/countries/br;cn;us;de/indicators/SP.POP.TOTL/?format=json&per_page=1000'
r = requests.get(url)
payload = r.json()
print(payload)

In [0]:
# payload[0] → metadata
# payload[1] → lista de registros
records = payload[1]

# 3. Crear DataFrame directamente desde la lista de dicts
df_raw = spark.createDataFrame(records)

# 4. Aplanar los structs anidados y castear columnas
df = df_raw.select(
    col("indicator.id").alias("indicator_id"),
    col("indicator.value").alias("indicator_name"),
    col("country.id").alias("country_id"),
    col("country.value").alias("country_name"),
    col("countryiso3code"),
    col("date").cast("int").alias("year"),
    col("value").cast("long").alias("population"),
    col("unit"),
    col("obs_status"),
    col("decimal").cast("int")
)

# 5. Verificar esquema y resultados
df.printSchema()
df.show(10, False)